In [1]:
import hail as hl
from hail.experimental import ManhattanPreprocessor
from hail.experimental import TileGenerator

In [2]:
hl.init()

Running on Apache Spark version 2.2.0
SparkUI available at http://10.1.7.107:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version devel-99a9111eb97f
NOTE: This is a beta version. Interfaces may change
  during the beta period. We recommend pulling
  the latest changes weekly.


In [5]:
colors = {}
for i in range(1,23):
    if i%2==0:
        colors[str(i)] = '#0e6d19'
    else:
        colors[str(i)] = "#000000"
colors['X'] = "#000000"

In [9]:
# add manhattan plotting data to mt
mt = hl.read_matrix_table('/Users/maccum/manhattan_data/raw/caffeine_mt')
mp = ManhattanPreprocessor(mt.locus, mt.phenotype, mt.p_value)
manhat_mt = mp.add_manhattan_data(colors)
manhat_mt.write('/Users/maccum/manhattan_data/with_plot_fields/caffeine_mt2', overwrite=True)

# read manhattan plotting data from disk and start tile generation
manhat_mt = hl.read_matrix_table('/Users/maccum/manhattan_data/with_plot_fields/caffeine_mt2')
tg = TileGenerator(manhat_mt,
                  dest='/Users/maccum/manhattan_data/plots/caffeine_plots_2',
                  regen=True)

tg.generate_tile_layer(phenotype='caffeine_consumption', zoom=2, new_log_file=True)


2018-08-24 17:55:32 Hail: INFO: wrote 10879 items in 16 partitions to /Users/maccum/manhattan_data/with_plot_fields/caffeine_mt2


Zoom level: 2 |██████████████████████████████████████████████████| 100.0% Complete


In [10]:
tg.generate_tile_layer(phenotype='caffeine_consumption',zoom=3)

Zoom level: 3 |██████████████████████████████████████████████████| 100.0% Complete


In [11]:
tg.generate_tile_layer(phenotype='caffeine_consumption',zoom=3)
tg.generate_tile_layer(phenotype='caffeine_consumption',zoom=4)
tg.generate_tile_layer(phenotype='caffeine_consumption',zoom=5)
tg.generate_tile_layer(phenotype='caffeine_consumption',zoom=6)
tg.generate_tile_layer(phenotype='caffeine_consumption',zoom=7)
tg.generate_tile_layer(phenotype='caffeine_consumption',zoom=8)

Zoom level: 3 |██████████████████████████████████████████████████| 100.0% Complete
Zoom level: 4 |██████████████████████████████████████████████████| 100.0% Complete
Zoom level: 5 |██████████████████████████████████████████████████| 100.0% Complete
Zoom level: 6 |██████████████████████████████████████████████████| 100.0% Complete
Zoom level: 7 |██████████████████████████████████████████████████| 100.0% Complete


FatalError: NullPointerException: null

Java stack trace:
java.lang.NullPointerException: null
	at is.hail.annotations.aggregators.RegionValueDownsampleAggregator.result(RegionValueDownsampleAggregator.scala:26)
	at is.hail.expr.ir.Interpret$$anonfun$apply$27$$anonfun$apply$28.apply(Interpret.scala:653)
	at is.hail.expr.ir.Interpret$$anonfun$apply$27$$anonfun$apply$28.apply(Interpret.scala:653)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
	at is.hail.expr.ir.Interpret$$anonfun$apply$27.apply(Interpret.scala:653)
	at is.hail.expr.ir.Interpret$$anonfun$apply$27.apply(Interpret.scala:647)
	at is.hail.utils.package$.using(package.scala:587)
	at is.hail.annotations.Region$.scoped(Region.scala:19)
	at is.hail.expr.ir.Interpret$.apply(Interpret.scala:647)
	at is.hail.expr.ir.Interpret$.apply(Interpret.scala:49)
	at is.hail.table.Table.aggregate(Table.scala:413)
	at is.hail.table.Tab

1